In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import requests
from bisect import bisect
import datetime

In [10]:
IHi_list = [50, 100, 200, 300, 400, 500]
ILo_list = [0, 51, 101, 201, 301, 401]
BPHi_list = [30, 60, 90, 120, 250, 500]
BPLo_list = [0, 31, 61, 91, 121, 251]

def pm25_aqi(Cp):
    index = bisect(BPLo_list, Cp) - 1
    aqi = ((IHi_list[index] - ILo_list[index]) / (BPHi_list[index] - BPLo_list[index])) * (Cp - BPLo_list[index]) + ILo_list[index]
    return aqi

In [14]:
def data_by_date(start_date, end_date, location_id):
    api_url = f"https://api.openaq.org/v2/measurements?date_from={start_date}&date_to={end_date}&limit=7000&sort=desc&parameter=pm25&location_id={location_id}&order_by=datetime"
    response = requests.get(api_url)
    print(response)

    results = response.json()["results"]
    df = pd.DataFrame(columns=['datetime', 'pm2.5'])

    for i, result in enumerate(results):
        datetime = result["date"]["local"]
        pm25 = result["value"]

        df.loc[i] = [datetime, pm25]

    df.datetime = pd.to_datetime(df.datetime)

    df['month'] = df['datetime'].dt.strftime('%B')
    df['day'] = df['datetime'].dt.strftime('%d')
    df['year'] = df['datetime'].dt.strftime('%Y')
    df['time'] = df['datetime'].dt.strftime('%H:%M:%S')

    df['sub_index'] = df['pm2.5'].map(pm25_aqi)

    return df

In [33]:
def month_wise_year_data(year, location_id):
    start_date = datetime.date(year, 1, 1).strftime('%Y-%m-%d')
    end_date = datetime.date(year, 12, 1).strftime('%Y-%m-%d')

    year_data = data_by_date(start_date, end_date, location_id)
    final_data = year_data.drop(['pm2.5', 'datetime', 'time'], axis=1).pivot_table(values=['day', 'month', 'year'], index='day', columns=['month', 'year'], aggfunc=np.mean)['sub_index'].reset_index()
    final_data.columns = [' '.join(col).rstrip() for col in final_data.columns.values]

    return final_data